In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.cluster import KMeans
pd.set_option("display.max_rows", 1000)    #設定最大能顯示1000rows
pd.set_option("display.max_columns", 1000)   #設定最大能顯示1000columns
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']  
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import tree
import os
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import  DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder       #轉字串為數字
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix


In [2]:
from google.colab import files
train = files.upload()
valid = files.upload()
test = files.upload()

Saving pokemon_train.csv to pokemon_train.csv


Saving pokemon_valid.csv to pokemon_valid.csv


Saving pokemon_test.csv to pokemon_test.csv


In [2]:
train_data = pd.read_csv("pokemon_train.csv", thousands=",")    #匯入csv檔，將千分位記號捨去
test_data = pd.read_csv("pokemon_test.csv", thousands=",")    #匯入csv檔，將千分位記號捨去
valid_data = pd.read_csv("pokemon_valid.csv", thousands=",") 

valid_data.head()

,Unnamed: 0,Type.1,Type.2,Total,HP,Attack,Defense,Sp..Atk,Sp..Def,Speed,Generation,Legendary
0,1,Water,NaN,630,79,103,120,135,115,78,1,False
1,2,Bug,NaN,195,45,30,35,20,20,45,1,False
2,3,Poison,NaN,438,60,85,69,65,79,80,1,False
3,4,Fairy,NaN,323,70,45,48,60,65,35,1,False
4,5,Bug,Poison,450,70,65,60,90,75,90,1,False


In [33]:
from sklearn.model_selection import train_test_split

train_x = train_data.drop(columns=['Type.1','Type.2'])   #刪除訓練集的Type1,Type2，保留特徵值
train_y = train_data[["Type.1"]]  #目標值

test_x = test_data[["Total","HP","Attack","Defense","Sp..Atk","Sp..Def","Speed","Generation","Legendary"]]
valid_x = valid_data[["Total","HP","Attack","Defense","Sp..Atk","Sp..Def","Speed","Generation","Legendary"]]
train_x.head()



,Unnamed: 0,Total,HP,Attack,Defense,Sp..Atk,Sp..Def,Speed,Generation,Legendary
0,1,318,45,49,49,65,65,45,1,False
1,2,405,60,62,63,80,80,60,1,False
2,3,525,80,82,83,100,100,80,1,False
3,4,625,80,100,123,122,120,80,1,False
4,5,309,39,52,43,60,50,65,1,False


In [5]:
# 將type1的分類特徵轉換為數字
le = LabelEncoder()   # 把每個類別 mapping 到某個整數
train_y["Type.1"] = le.fit_transform(train_y["Type.1"])
train_x["Legendary"] = le.fit_transform(train_x["Legendary"])
test_x["Legendary"] = le.fit_transform(test_x["Legendary"])
valid_x["Legendary"] = le.fit_transform(test_x["Legendary"])

train_y["Type.1"].head()
train_x["Legendary"].head()
train_y.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Type.1
0,9
1,9
2,9
3,9
4,6


In [12]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train_x, train_y, random_state=60)  ## 3.劃分訓練集和驗證集

# train_test_split是交叉驗證中常用的函式，功能是從樣本中隨機的按比例選取train_data和test_data
# random_state是隨機數種子，是該組隨機數的編號，在需要重複試驗的時候，保證得到一組一樣的隨機數。




In [13]:
def entropy_node_tree(name, num):
    model = DecisionTreeClassifier(criterion=name, max_leaf_nodes = num)   # Entropy node
    model.fit(X_train, Y_train)
    yyyyy = model.predict(X_valid)
    print("正確率為", metrics.accuracy_score(yyyyy, Y_valid))
for i in range(2,10):
    entropy_node_tree("entropy", i)

正確率為 0.18285714285714286
正確率為 0.1657142857142857
正確率為 0.18857142857142858
正確率為 0.2057142857142857
正確率為 0.21714285714285714
正確率為 0.21142857142857144
正確率為 0.2057142857142857
正確率為 0.2057142857142857


In [14]:
def entropy_depth_tree(num):
    model = DecisionTreeClassifier(criterion="entropy", max_depth= num)   # Entropy depth
    model.fit(X_train, Y_train)
    yyyyy = model.predict(X_valid)
    print("正確率為", metrics.accuracy_score(Y_valid, yyyyy))
for i in range(2,10):
    entropy_depth_tree(i)

正確率為 0.17714285714285713
正確率為 0.21714285714285714
正確率為 0.2057142857142857
正確率為 0.18857142857142858
正確率為 0.19428571428571428
正確率為 0.19428571428571428
正確率為 0.18285714285714286
正確率為 0.16


In [15]:
def gini_node_tree(num):
    model = DecisionTreeClassifier(criterion="gini", max_leaf_nodes = num)   # Gini node
    model.fit(X_train, Y_train)
    yyyyy = model.predict(X_valid)
    print("正確率為", metrics.accuracy_score(Y_valid, yyyyy))
for i in range(2,10):
    gini_node_tree(i)

正確率為 0.22857142857142856
正確率為 0.22857142857142856
正確率為 0.22857142857142856
正確率為 0.2342857142857143
正確率為 0.24
正確率為 0.24571428571428572
正確率為 0.24571428571428572
正確率為 0.2057142857142857


In [27]:
def gini_depth_tree(num):
    model = DecisionTreeClassifier(criterion="gini", max_depth= num)  # Gini depth
    model.fit(X_train, Y_train)
    yyyyy = model.predict(X_valid)
    print("正確率為", metrics.accuracy_score(Y_valid, yyyyy))
for i in range(2,10):
    gini_depth_tree(i)

正確率為 0.22857142857142856
正確率為 0.21142857142857144
正確率為 0.2
正確率為 0.24571428571428572
正確率為 0.26285714285714284
正確率為 0.22857142857142856
正確率為 0.21714285714285714
正確率為 0.13714285714285715


In [19]:
model = DecisionTreeClassifier(criterion="gini", max_depth = 6)  
model.fit(X_train, Y_train) 
yyyyy = model.predict(X_valid)
print("正確率為", metrics.accuracy_score(Y_valid, yyyyy))  
confuse_matrix = confusion_matrix(Y_valid, yyyyy)                     # 混淆矩陣
print(confuse_matrix)

正確率為 0.25142857142857145
[[ 7  0  0  2  0  0  0  0  0  1  0  0  2  0  0  0  0  2]
 [ 1  0  3  2  0  0  1  0  0  2  0  0  1  0  0  1  0  0]
 [ 1  0  3  0  0  0  0  0  0  0  0  0  2  0  0  0  0  1]
 [ 0  0  0  3  0  0  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  1  0  1  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  4  0  0  3  0  0  1  0  0  1  0  0  1  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  1  0  0  1  2  0  2  0  0  0  0  0]
 [ 2  0  1  1  0  0  1  0  0  3  1  0  1  0  0  0  0  2]
 [ 0  0  1  0  0  1  0  0  0  0  0  0  5  0  1  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 5  0  1  2  0  3  2  0  0  1  0  0 12  0  1  0  0  1]
 [ 0  0  0  0  0  0  1  0  0  2  0  0  0  0  0  0  0  1]
 [ 4  0  0  5  0  0  0  0  0  2  0  0  0  0  6  1  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  1  0  2  0  1  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  1  0  0  2  0  0  0  0  1]
 [ 4  

In [35]:
def gini_depth_forest(num):
    model = RandomForestClassifier(criterion="gini", max_depth= num)  # Gini depth 隨機森林
    model.fit(X_train, Y_train)
    yyyyy = model.predict(X_valid)
    print("正確率為", metrics.accuracy_score(yyyyy, Y_valid))
for i in range(2,10):
    gini_depth_tree(i)

正確率為 0.22857142857142856
正確率為 0.21142857142857144
正確率為 0.2
正確率為 0.24
正確率為 0.25142857142857145
正確率為 0.22857142857142856
正確率為 0.22285714285714286
正確率為 0.14857142857142858


In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, random_state=60)  ## 3.劃分數據集

model_tree =  DecisionTreeClassifier(criterion='gini',max_depth=6)   # 決策樹 使用測試集
model_tree.fit(X_train, Y_train)
aaa = model_tree.predict(X_test)
print("決策樹正確率為", metrics.accuracy_score(aaa, Y_test))


model_forest =  RandomForestClassifier(criterion='gini',max_depth=6)   # 隨機森林 使用測試集
model_forest.fit(X_train, Y_train)
aaa = model_forest.predict(X_test)
print("隨機森林正確率為", metrics.accuracy_score(aaa, Y_test))


決策樹正確率為 0.26285714285714284


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


隨機森林正確率為 0.32
